# Lab3 - Second part: Machine learning in Spark
## Task 1 - Toxicology prediction with SVM

Write a Spark program that builds a toxicology prediction model form the  toxicity.txt dataset, using Support Vector Machines (SVM). In order to evaluate the model, use 80% of the dataset as training set, and save the remaining 20% for testing. As evaluation metric you can use the area under the ROC curve.

In [ ]:
TODO

## What is the limitation of Hadoop that makes Machine Learning infeasible? How does Spark overcome this limitation?

TODO

## How would you rate the model that you obtained in this task? Is that a good model?

TODO